In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 218.4MB 61kB/s 
     |████████████████████████████████| 204kB 44.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.6-py2.py3-none-any.whl size=218814406 sha256=10328abeabe8769d292941ce8cb0dc5376b249841ba9671d54c1f2e5f3e1d711
  Stored in directory: /root/.cache/pip/wheels/1e/5e/6a/17e906c94ec7246f260330a66e44a06a0809033ba2738a74a8
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 43 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [0]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

#  Organización de Datos 75.06 . Primer Cuatrimestre de 2019. Examen parcial, primera oportunidad:
1) El servicio meteorológico registra datos del
tiempo para todas las ciudades donde posee
una base de medición.
Esta información se encuentra en dos RDD.
En el primero se tiene información de las
bases de medición: (ID_BASE, NOMBRE,
PCIA, CIUDAD, LAT, LON).
El segundo posee información sobre las
mediciones en sí: (TIMESTAMP, ID_BASE,
TEMPERATURA, HUMEDAD, PRESIÓN,
DIRECCIÓN VIENTO, VELOCIDAD VIENTO).
Se desea obtener un reporte para las bases de
la Provincia de Buenos Aires. El mismo debe
informar los ID y nombre de bases de
medición que hayan registrado una variación
de temperatura promedio mensual mayor al
30% en el año 2018 (dada la temperatura
promedio de un mes, esta se debe comparar
con el promedio del mes anterior, para
determinar la variación porcentual).
Resolver utilizando el API de RDD de
PySpark, dando el reporte en un RDD. (***) (
15pts)


In [0]:
# (id_base,nombre,pcia,ciudad,lat,lon)
bases = [
    (1, 'base1', 'Buenos Aires', 'San Fernando', -1432, 93482),
    (2, 'base2', 'Mendoza', 'San Rafael', -32432, 43243),
    (3, 'base3', 'Mendoza', 'Las Leñas', -32313, 354345),
    (4, 'base4', 'Buenos Aires', 'CABA', -123123, 432423),
    (5, 'base5', 'Neuquen', 'Bariloche', -634534, 564563),
    (6, 'base6', 'Buenos Aires', 'Merlo', -453453,57454),
    (7, 'base7', 'Buenos Aires', 'Avellaneda', -2493284, -3428234),
    (8, 'base8', 'Buenos Aires', 'Tigre', -32432, 654654),
    (9, 'base9', 'Buenos Aires', 'Quilmes', -98324, -3284723),
    (10, 'base10', 'Misiones', 'San Ignacio', -654645, 564345)
]

# (timestamp, id_base, temperatura, humedad, presion, direccion viento, velocidad viento)
mediciones = [
    ('2018-03-04', 1, 25, 96, 300, 'NO', 47),
    ('2017-12-03', 1, 25, 96, 300, 'NO', 47),
    ('2018-04-05', 1, 15, 96, 300, 'NO', 47),
    ('2018-05-12', 1, 18, 96, 300, 'NO', 47),
    ('2018-05-21', 1, 23, 96, 300, 'NO', 47),
    ('2018-01-04', 1, 38, 96, 300, 'NO', 47),
    ('2018-02-29', 1, 28, 96, 300, 'NO', 47),
    ('2018-06-14', 1, 9, 96, 300, 'NO', 47),
    ('2018-07-12', 1, 5, 96, 300, 'NO', 47),
    ('2018-08-31', 1, 19, 96, 300, 'NO', 47),
    ('2018-09-14', 1, 20, 96, 300, 'NO', 47),
    ('2019-03-12', 1, 23, 96, 300, 'NO', 47),
    ('2019-04-15', 1, 16, 96, 300, 'NO', 47),
    ('2018-07-12', 2, 5, 96, 300, 'NO', 47),
    ('2018-07-31', 2, 7, 96, 300, 'NO', 47),
    ('2018-09-14', 4, 26, 96, 300, 'NO', 47),
    ('2018-08-12', 4, 13, 96, 300, 'NO', 47),
    ('2019-04-15', 4, 16, 96, 300, 'NO', 47)
]

In [0]:
basesRDD = sc.parallelize(bases)
medicionesRDD = sc.parallelize(mediciones)

In [130]:
basesRDD.collect()

[(1, 'base1', 'Buenos Aires', 'San Fernando', -1432, 93482),
 (2, 'base2', 'Mendoza', 'San Rafael', -32432, 43243),
 (3, 'base3', 'Mendoza', 'Las Leñas', -32313, 354345),
 (4, 'base4', 'Buenos Aires', 'CABA', -123123, 432423),
 (5, 'base5', 'Neuquen', 'Bariloche', -634534, 564563),
 (6, 'base6', 'Buenos Aires', 'Merlo', -453453, 57454),
 (7, 'base7', 'Buenos Aires', 'Avellaneda', -2493284, -3428234),
 (8, 'base8', 'Buenos Aires', 'Tigre', -32432, 654654),
 (9, 'base9', 'Buenos Aires', 'Quilmes', -98324, -3284723),
 (10, 'base10', 'Misiones', 'San Ignacio', -654645, 564345)]

In [131]:
medicionesRDD.collect()

[('2018-03-04', 1, 25, 96, 300, 'NO', 47),
 ('2017-12-03', 1, 25, 96, 300, 'NO', 47),
 ('2018-04-05', 1, 15, 96, 300, 'NO', 47),
 ('2018-05-12', 1, 18, 96, 300, 'NO', 47),
 ('2018-05-21', 1, 23, 96, 300, 'NO', 47),
 ('2018-01-04', 1, 38, 96, 300, 'NO', 47),
 ('2018-02-29', 1, 28, 96, 300, 'NO', 47),
 ('2018-06-14', 1, 9, 96, 300, 'NO', 47),
 ('2018-07-12', 1, 5, 96, 300, 'NO', 47),
 ('2018-08-31', 1, 19, 96, 300, 'NO', 47),
 ('2018-09-14', 1, 20, 96, 300, 'NO', 47),
 ('2019-03-12', 1, 23, 96, 300, 'NO', 47),
 ('2019-04-15', 1, 16, 96, 300, 'NO', 47),
 ('2018-07-12', 2, 5, 96, 300, 'NO', 47),
 ('2018-07-31', 2, 7, 96, 300, 'NO', 47),
 ('2018-09-14', 4, 26, 96, 300, 'NO', 47),
 ('2018-08-12', 4, 13, 96, 300, 'NO', 47),
 ('2019-04-15', 4, 16, 96, 300, 'NO', 47)]

In [132]:
#primero filtro bases de buenos aires
basesRDD = basesRDD.filter(lambda x: x[2] == 'Buenos Aires')
basesRDD.collect()

[(1, 'base1', 'Buenos Aires', 'San Fernando', -1432, 93482),
 (4, 'base4', 'Buenos Aires', 'CABA', -123123, 432423),
 (6, 'base6', 'Buenos Aires', 'Merlo', -453453, 57454),
 (7, 'base7', 'Buenos Aires', 'Avellaneda', -2493284, -3428234),
 (8, 'base8', 'Buenos Aires', 'Tigre', -32432, 654654),
 (9, 'base9', 'Buenos Aires', 'Quilmes', -98324, -3284723)]

In [133]:
#filtro mediciones de 2018
medicionesRDD = medicionesRDD.filter(lambda x: '2018' in x[0])
medicionesRDD.collect()

[('2018-03-04', 1, 25, 96, 300, 'NO', 47),
 ('2018-04-05', 1, 15, 96, 300, 'NO', 47),
 ('2018-05-12', 1, 18, 96, 300, 'NO', 47),
 ('2018-05-21', 1, 23, 96, 300, 'NO', 47),
 ('2018-01-04', 1, 38, 96, 300, 'NO', 47),
 ('2018-02-29', 1, 28, 96, 300, 'NO', 47),
 ('2018-06-14', 1, 9, 96, 300, 'NO', 47),
 ('2018-07-12', 1, 5, 96, 300, 'NO', 47),
 ('2018-08-31', 1, 19, 96, 300, 'NO', 47),
 ('2018-09-14', 1, 20, 96, 300, 'NO', 47),
 ('2018-07-12', 2, 5, 96, 300, 'NO', 47),
 ('2018-07-31', 2, 7, 96, 300, 'NO', 47),
 ('2018-09-14', 4, 26, 96, 300, 'NO', 47),
 ('2018-08-12', 4, 13, 96, 300, 'NO', 47)]

In [134]:
# mapeo ambos RDDs
basesRDD_aux = basesRDD.map(lambda x: (x[0], x[1]))
medicionesRDD_aux = medicionesRDD.map(lambda x: (x[1], (x[0],x[2])))
# joineo ahora asi me quedan las mediciones de buenos aires
medsRDD = basesRDD_aux.join(medicionesRDD_aux)
medsRDD.collect()

[(4, ('base4', ('2018-09-14', 26))),
 (4, ('base4', ('2018-08-12', 13))),
 (1, ('base1', ('2018-03-04', 25))),
 (1, ('base1', ('2018-04-05', 15))),
 (1, ('base1', ('2018-05-12', 18))),
 (1, ('base1', ('2018-05-21', 23))),
 (1, ('base1', ('2018-01-04', 38))),
 (1, ('base1', ('2018-02-29', 28))),
 (1, ('base1', ('2018-06-14', 9))),
 (1, ('base1', ('2018-07-12', 5))),
 (1, ('base1', ('2018-08-31', 19))),
 (1, ('base1', ('2018-09-14', 20)))]

In [135]:
# calculo la temperatura promedio por mes, para eso pongo como clave el mes y año 
medsRDD = medsRDD.map(lambda x: ((x[1][1][0].split('-')[0], x[1][1][0].split('-')[1],x[0],x[1][0]),(x[1][1][1], 1)))
medsRDD.collect()

[(('2018', '09', 4, 'base4'), (26, 1)),
 (('2018', '08', 4, 'base4'), (13, 1)),
 (('2018', '03', 1, 'base1'), (25, 1)),
 (('2018', '04', 1, 'base1'), (15, 1)),
 (('2018', '05', 1, 'base1'), (18, 1)),
 (('2018', '05', 1, 'base1'), (23, 1)),
 (('2018', '01', 1, 'base1'), (38, 1)),
 (('2018', '02', 1, 'base1'), (28, 1)),
 (('2018', '06', 1, 'base1'), (9, 1)),
 (('2018', '07', 1, 'base1'), (5, 1)),
 (('2018', '08', 1, 'base1'), (19, 1)),
 (('2018', '09', 1, 'base1'), (20, 1))]

In [136]:
medsRDD = medsRDD.reduceByKey(lambda x,y: ((x[0]+y[0])/(x[1]+y[1]))) #calculo del promedio de temperatura por base
medsRDD_aux = medsRDD.map(lambda x: (x[0], x[1][0]/x[1][1]) if ((type(x[1])==tuple)) else (x[0],x[1])).sortByKey()
medsRDD_aux.collect()

[(('2018', '01', 1, 'base1'), 38.0),
 (('2018', '02', 1, 'base1'), 28.0),
 (('2018', '03', 1, 'base1'), 25.0),
 (('2018', '04', 1, 'base1'), 15.0),
 (('2018', '05', 1, 'base1'), 20.5),
 (('2018', '06', 1, 'base1'), 9.0),
 (('2018', '07', 1, 'base1'), 5.0),
 (('2018', '08', 1, 'base1'), 19.0),
 (('2018', '08', 4, 'base4'), 13.0),
 (('2018', '09', 1, 'base1'), 20.0),
 (('2018', '09', 4, 'base4'), 26.0)]

In [0]:
def comparar(x):
    tempAnterior = 0
    newList = []
    for registro in x[1]:
        if (((registro[0]-tempAnterior)/registro[0]) > 0.3) or (((registro[0]-tempAnterior)/registro[0]) < -0.3):
                newReg = (((registro[0]-tempAnterior)/registro[0]), registro[3])
                newList.append(newReg)
        tempAnterior = registro[0]
    return (x[0], newList)

In [147]:
medsFinal = medsRDD_aux.map(lambda x: (x[0][2], (x[1],x[0][0],x[0][1],x[0][3]))).groupByKey().map(lambda x: (x[0], list(x[1])))\
.map(comparar)
medsFinal.collect()

[(4, [(1.0, 'base4'), (0.5, 'base4')]),
 (1,
  [(1.0, 'base1'),
   (-0.35714285714285715, 'base1'),
   (-0.6666666666666666, 'base1'),
   (-1.2777777777777777, 'base1'),
   (-0.8, 'base1'),
   (0.7368421052631579, 'base1')])]